In [1]:
import pandas as pd
import seaborn as sns
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, Perceptron, SGDRegressor, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm
from sklearn.svm import SVC
import scipy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import pickle
from joblib import dump, load
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

In [2]:
data = pd.read_csv("/home/mike/python_files/ML_services/data/NHANES_age_prediction.csv")
data["age_group"]
# data = pd.read_csv("/home/mike/python_files/ML_services/data/NHANES_age_prediction.csv").drop("age_group", axis=1)
data.describe()

,SEQN,RIDAGEYR,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN
count,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000
mean,78691.853819,41.795874,1.511414,1.822651,27.955180,99.553117,2.016242,114.978929,11.834794
std,2921.365151,20.156111,0.499979,0.398918,7.248962,17.889834,0.185556,47.061239,9.718812
min,73564.000000,12.000000,1.000000,1.000000,14.500000,63.000000,1.000000,40.000000,0.140000
25%,76171.750000,24.000000,1.000000,2.000000,22.800000,91.000000,2.000000,87.000000,5.860000
50%,78749.000000,41.000000,2.000000,2.000000,26.800000,97.000000,2.000000,105.000000,9.040000
75%,81213.500000,58.000000,2.000000,2.000000,31.200000,104.000000,2.000000,130.000000,14.440000
max,83727.000000,80.000000,2.000000,7.000000,70.100000,405.000000,3.000000,604.000000,102.290000


In [3]:
le = LabelEncoder()

In [4]:
le.fit(data["age_group"])
data["age_group"] = le.transform(data["age_group"])
data

,SEQN,age_group,RIDAGEYR,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN
0,73564.0,0,61.0,2.0,2.0,35.7,110.0,2.0,150.0,14.91
1,73568.0,0,26.0,2.0,2.0,20.3,89.0,2.0,80.0,3.85
2,73576.0,0,16.0,1.0,2.0,23.2,89.0,2.0,68.0,6.14
3,73577.0,0,32.0,1.0,2.0,28.9,104.0,2.0,84.0,16.15
4,73580.0,0,38.0,2.0,1.0,35.9,103.0,2.0,81.0,10.92
...,...,...,...,...,...,...,...,...,...,...
2273,83711.0,0,38.0,2.0,2.0,33.5,100.0,2.0,73.0,6.53
2274,83712.0,0,61.0,1.0,2.0,30.0,93.0,2.0,208.0,13.02
2275,83713.0,0,34.0,1.0,2.0,23.7,103.0,2.0,124.0,21.41
2276,83718.0,0,60.0,2.0,2.0,27.4,90.0,2.0,108.0,4.99


In [5]:
data.drop(["RIDAGEYR", "SEQN"], axis=1).describe()

,age_group,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN
count,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000
mean,0.159789,1.511414,1.822651,27.955180,99.553117,2.016242,114.978929,11.834794
std,0.366491,0.499979,0.398918,7.248962,17.889834,0.185556,47.061239,9.718812
min,0.000000,1.000000,1.000000,14.500000,63.000000,1.000000,40.000000,0.140000
25%,0.000000,1.000000,2.000000,22.800000,91.000000,2.000000,87.000000,5.860000
50%,0.000000,2.000000,2.000000,26.800000,97.000000,2.000000,105.000000,9.040000
75%,0.000000,2.000000,2.000000,31.200000,104.000000,2.000000,130.000000,14.440000
max,1.000000,2.000000,7.000000,70.100000,405.000000,3.000000,604.000000,102.290000


In [6]:
# train_X, test_X, train_y, test_y = train_test_split(data.drop(["age_group", "RIDAGEYR"], axis=1), data["age_group"], test_size=0.2, stratify=data["age_group"])
train_X, test_X, train_y, test_y = train_test_split(data.drop(["RIDAGEYR", "SEQN"], axis=1), data["RIDAGEYR"], test_size=0.2, stratify=data["RIDAGEYR"])
mms = MinMaxScaler()
mms.fit(pd.concat((train_X,test_X)).sort_index())
train_X = pd.DataFrame(mms.transform(train_X))
test_X = pd.DataFrame(mms.transform(test_X))
train_X

,0,1,2,3,4,5,6,7
0,0.0,1.0,0.166667,0.151079,0.102339,0.5,0.063830,0.122173
1,1.0,0.0,0.166667,0.347122,0.102339,0.5,0.161348,0.083603
2,0.0,0.0,0.166667,0.393885,0.096491,0.5,0.118794,0.284288
3,1.0,1.0,0.166667,0.296763,0.099415,0.5,0.255319,0.044053
4,0.0,0.0,0.166667,0.064748,0.052632,0.5,0.125887,0.000000
...,...,...,...,...,...,...,...,...
1817,0.0,0.0,0.166667,0.210432,0.128655,0.5,0.074468,0.147332
1818,0.0,1.0,0.000000,0.624101,0.099415,0.5,0.141844,0.094958
1819,0.0,0.0,0.166667,0.219424,0.102339,0.5,0.154255,0.142242
1820,0.0,1.0,0.166667,0.190647,0.061404,0.5,0.079787,0.052766


In [7]:
mms_pickle = pickle.dumps(mms)
dump(mms_pickle, '/home/mike/python_files/ML_services/ml_models/MinMaxScaler_pickle.joblib') 

['/home/mike/python_files/ML_services/ml_models/MinMaxScaler_pickle.joblib']

In [8]:
lr = LinearRegression()
lr.fit(train_X, train_y)
display(mean_squared_error(test_y, lr.predict(test_X)))
display(mean_absolute_error(test_y, lr.predict(test_X)))
lr_pickle = pickle.dumps(lr)
dump(
    lr_pickle,
    "/home/mike/python_files/ML_services/ml_models/LinearRegression_pickle.joblib",
)

186.10668163306642

11.396605244259193

['/home/mike/python_files/ML_services/ml_models/LinearRegression_pickle.joblib']

In [9]:
params = {
    "penalty": ("l2", "l1", "elasticnet"),
    "alpha": [0.01, 0.001, 0.0001, 0.00001],
}
grcv = GridSearchCV(estimator=Perceptron(), param_grid=params)
grcv.fit(train_X, train_y)
display(mean_squared_error(test_y, grcv.predict(test_X)))
display(mean_absolute_error(test_y, grcv.predict(test_X)))
best_perceptron_pickle = pickle.dumps(grcv.best_estimator_)
dump(
    best_perceptron_pickle,
    "/home/mike/python_files/ML_services/ml_models/best_perceptron_pickle.joblib",
)

235.27850877192984

12.677631578947368

['/home/mike/python_files/ML_services/ml_models/best_perceptron_pickle.joblib']

In [10]:
params = {
    "n_neighbors": list(range(2, 15)),
    "weights": ("uniform", "distance"),
}
grcv = GridSearchCV(estimator=KNeighborsRegressor(), param_grid=params)
grcv.fit(train_X, train_y)
display(mean_squared_error(test_y, grcv.predict(test_X)))
display(mean_absolute_error(test_y, grcv.predict(test_X)))
kneighborsregressor_pickle = pickle.dumps(grcv.best_estimator_)
dump(
    kneighborsregressor_pickle,
    "/home/mike/python_files/ML_services/ml_models/KNeighborsRegressor.joblib",
)

155.36563182970073

10.004615268192216

['/home/mike/python_files/ML_services/ml_models/KNeighborsRegressor.joblib']

In [11]:
params = {
    "penalty": ("l2", "l1", "elasticnet"),
    "alpha": [0.01, 0.001, 0.0001, 0.00001],
}
grcv = GridSearchCV(estimator=SGDRegressor(), param_grid=params)
grcv.fit(train_X, train_y)
display(mean_squared_error(test_y, grcv.predict(test_X)))
display(mean_absolute_error(test_y, grcv.predict(test_X)))
SGDRegressor_pickle = pickle.dumps(grcv.best_estimator_)
dump(
    SGDRegressor_pickle,
    "/home/mike/python_files/ML_services/ml_models/SGDRegressor.joblib",
)

188.75665250574013

11.513091170564593

['/home/mike/python_files/ML_services/ml_models/SGDRegressor.joblib']

In [12]:
params = {
    "max_depth": list(range(2, 15, 2)),
    "min_samples_split": list(range(2, 15, 2)),
    "min_samples_leaf": list(range(2, 15, 2)),
    "criterion": (
        "squared_error",
        "friedman_mse",
        "absolute_error",
        "poisson",
    ),
}
grcv = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=params)
grcv.fit(train_X, train_y)
display(mean_squared_error(test_y, grcv.predict(test_X)))
display(mean_absolute_error(test_y, grcv.predict(test_X)))
dtr_pickle = pickle.dumps(grcv.best_estimator_)
dump(
    dtr_pickle,
    "/home/mike/python_files/ML_services/ml_models/DecisionTreeRegressor.joblib",
)

183.0506160536092

10.707998188918685

['/home/mike/python_files/ML_services/ml_models/DecisionTreeRegressor.joblib']